In [11]:
import torch
import torch.nn as nn
import string
import random
import sys
import unidecode
from torch.utils.tensorboard import SummaryWriter

In [12]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [13]:
# Get chars from string.printable
all_chars = string.printable
n_chars = len(all_chars)

In [14]:
import random

# Read the lines from the original file
with open('dataset/babynames.txt', 'r') as file:
    lines = file.readlines()

# Shuffle the lines
random.shuffle(lines)

# Write the shuffled lines to a new file
with open('dataset/shuffled_babynames.txt', 'w') as file:
    file.writelines(lines)


file = unidecode.unidecode(open('dataset/shuffled_babynames.txt').read())

In [ ]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.embed = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden, cell):
        out = self.embed(x)
        out, (hidden, cell) = self.lstm(out.unsqueeze(1), (hidden, cell))
        out = self.fc(out.reshape(out.shape[0], -1))
        return out, (hidden,cell)
    
    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(DEVICE)
        cell = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(DEVICE)
        return hidden, cell

In [ ]:
class Generator():
    def __init__(self):
        self.chunk_lenght = 250